<a href="https://colab.research.google.com/github/tmdgusya/-MD-/blob/main/python_unittest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytest tutorial

모의 객체(Mock object) 사용하기.

- 알아두면 좋은점 PYTEST 에서는 **PATCHING** 이라는 단어가 자주 나오는데 이는 테스트 기간동안만 임시로 교체한다는 뉘앙스를 지니고 있음

In [1]:
class ProductClass:
  def method(self):
    return self.something(1, 2, 3)
  def something(self, a, b, c):
    print(a, b, c)

In [2]:
from unittest.mock import MagicMock

real = ProductClass()
real.something = MagicMock()

real.method()

real.something.assert_called_once_with(1, 2, 3)

## 호출추적

기본적으로 `Mock` 객체는 아래와 같이 호출된 모든 메소드를 기록함.

In [18]:
mock = MagicMock()
mock.method()
mock.attribute.method(10, x=53)
mock.mock_calls

[call.method(), call.attribute.method(10, x=53)]

따라서 아래와 같은 방법으로 **특정 인자와 함께** 메소드가 호출되었는지 안되었는지 테스트 해볼수 있음.

In [19]:
mock.method.assert_called_once_with()
mock.attribute.method.assert_called_once_with(10, x=53)

만약에 attribute 가 중복 호출되는 경우에는 어떻게 체크할까?

In [20]:
mock.attribute.method(10, x=43)
mock.mock_calls

[call.method(),
 call.attribute.method(10, x=53),
 call.attribute.method(10, x=43)]

In [22]:
mock.attribute.method.assert_called_with(10, x=43)
mock.attribute.method.assert_called_with(10, x=53)

AssertionError: expected call not found.
Expected: method(10, x=53)
  Actual: method(10, x=43)

위와 같은 방법으로 테스트 하게 되면 에러가 발생하게 됨. 그 이유는 `assert_called_with` 의 주석을 살펴보면 **마지막 호출이 특정 인자와 함께 호출되었는지 검증(assert that the last call was made with the specified arguments)** 라고 적혀있음.

따라서 위와 같은 상황에서는 라이브러리를 살펴보면 `assert_any_call` 을 사용하는 것이 합당해 보임. `assert_any_call` 은 **모의 객체가 특정 인자와 함께 호출되었는지 검증(assert the mock has been called with the specified arguments)** 이기 때문에 이 메소드를 이용해야 함.

In [23]:
mock.attribute.method.assert_any_call(10, x=43)
mock.attribute.method.assert_any_call(10, x=53)

## 반환값과 속성값 설정

이 부분은 위에서 봐서 익숙할 수 있음. 그리고 매우 쉽고 간단하다.

In [40]:
mock = MagicMock()
mock.get_name.return_value = "roach"
mock.get_name()

'roach'

In [41]:
mock.name = "roach"
mock.name

'roach'

## 예외 발생시키기

이번에는 예외를 발생시키는 개념을 알아보겠습니다. 만약 `AService` 에서 `ARepository` 를 이용하고 있는데, `Arepository` 의 `find_by_id` 에서 예외가 나는 경우를 한번 만들어보도록 하겠습니다.

In [42]:
class ARepository:
  def __init__(self, db_connection) -> None:
    self.db_connection = db_connection

  def find_by_id(self):
    return self.db_connection.find_by_id()

class AService:

  def __init__(self, repository) -> None:
    self.repository = repository

  def find_by_id(self):
    return self.repository.find_by_id()

In [44]:
mock_repository = MagicMock()
mock_repository.find_by_id.side_effect = Exception("error") # set exception
service = AService(repository=mock_repository)

try:
  service.find_by_id()
except Exception as e:
  print(e) # error

error


## 순차적인 동일함수 호출 모킹

`side_effect` 는 순차적으로 실행되는 동일함수를 모킹할때도 동일하게 이용할 수 있습니다.

## patch

위에서 patching 에 대해 이야기 했듯이 unittest 에서는 `patch` 라는 함수를 제공한다. 아래 예시와 같이 `request` 의 `get` 함수도 모킹가능하다. 실제 우리가 자주 사용하는 `util` 함수 혹은 함수의 인자로 받지 않는 것들, 혹은 우리가 통제하지 않는 코드 모듈을 모킹할때 이용하면 편하다.


In [34]:
import requests

def get_naver():
  response = requests.get("https://www.naver.com")
  print(f"response: {response.status_code}")
  if response.status_code == 200:
    return "success"
  else:
    return "failure"

In [38]:
from unittest.mock import patch

with patch("requests.get") as mock:
  print(f"Instance: {mock}") # Instance: <MagicMock name='abs.instance' id='139804934001680'>
  mock_response = MagicMock()
  mock_response.status_code = 200
  mock.return_value = mock_response

  assert get_naver() == "success"

Instance: <MagicMock name='get' id='139804874593424'>
response: 200


In [49]:
from collections import namedtuple

Product = namedtuple('Product', ['name', 'price'])
product1 = Product('apple', 1000)
product2 = Product('banana', 2000)
product3 = Product('orange', 3000)

products = [product1, product2, product3]

products

[Product(name='apple', price=1000),
 Product(name='banana', price=2000),
 Product(name='orange', price=3000)]

side_effect 로 동일한 함수를 연속적으로 호출했을때 순차적으로 어떻게 반응해야 하는지를 정의하기 위해서는 이터러블한 객체로 전달해주는 방법이 존재한다. 아래 예시와 함께 살펴보자.

In [53]:
mock = MagicMock()
mock.is_exist.side_effect = [False, True, True]
for product in products:
  print(product)

  if mock.is_exist(product):
    continue

  print(f"filtered product: {product}")

Product(name='apple', price=1000)
filtered product: Product(name='apple', price=1000)
Product(name='banana', price=2000)
Product(name='orange', price=3000)


위와 같이 순차적으로 제시하는 방법도 있지만, 더욱 명확하게 하기 위해서는 특정 인자가 들어왔을때 특정값을 리턴하도록 하는 방법도 존재합니다.

In [67]:
from unittest.mock import MagicMock

# 1. 키를 (키, 값) 쌍의 정렬된 튜플로 변경
expected_results = {
    (('x', 1), ('y', 2)): 3,
    (('x', 4), ('y', 9)): 13
}

def side_effect_func(*args, **kwargs):
    print(f"args: {args}, kwargs: {kwargs}")
    if kwargs and isinstance(kwargs, dict):
        input_dict = kwargs
        # 입력 딕셔너리를 키 형식(정렬된 튜플)으로 변환
        key = tuple(sorted(input_dict.items()))
        if key in expected_results:
            return expected_results[key]
    # 예상치 못한 입력 처리 (옵션)
    raise KeyError(f"Arguments {args} not found in expected side effect results")

mock = MagicMock()
mock.add.side_effect = side_effect_func

# 테스트
result1 = mock.add(x=1, y=2) # 순서가 달라도 정렬되므로 동일한 키로 인식
result2 = mock.add(x=4, y=9)

print(f"Result 1: {result1}")
print(f"Result 2: {result2}")

assert result1 == 3
assert result2 == 13

mock.add.assert_any_call(x=1, y=2)
mock.add.assert_any_call(x=4, y=9)

print("Test passed!")

args: (), kwargs: {'x': 1, 'y': 2}
args: (), kwargs: {'x': 4, 'y': 9}
Result 1: 3
Result 2: 13
Test passed!


### patch.object

위에서 우리가 배운 `patch` 는 여러가지 편의 데코레이터들을 제공하는데 그 중 하나인 [`patch.object`](https://docs.python.org/ko/3/library/unittest.mock.html#unittest.mock.patch.object) 에 대해서 다뤄보자. `patch.object` 는 주로 클래스 혹은 모듈의 속성(attribute) 를 패치할때 이용한다.



In [73]:
class Someclass:

  attribute = 1

  def __init__(self) -> None:
    pass

In [87]:
from unittest.mock import patch, sentinel

# 원본 속성값을 original 에 저장
original = Someclass.attribute
print(f"[Outer Test] attribute value of Someclass.attribute: {Someclass.attribute}")
print(f"[Outer Test] attribute value of original: {original}")

@patch.object(Someclass, 'attribute', sentinel.attribute)
# patch object 를 통해서 attribute 바꾸기 (sentinel.attribute 로)
def test():
    print(f"[In test] attribute value of Someclass.attribute: {Someclass.attribute}")
    print(f"[In test] attribute value of Someclass.attribute: {sentinel.attribute}")
    print(f"[In test] attribute value of original: {original}")
    assert Someclass.attribute == sentinel.attribute

test()
print(f"[Outer Test] attribute value of Someclass.attribute: {Someclass.attribute}")
print(f"[Outer Test] attribute value of original: {original}")
assert Someclass.attribute == original

[Outer Test] attribute value of Someclass.attribute: 1
[Outer Test] attribute value of original: 1
[In test] attribute value of Someclass.attribute: sentinel.attribute
[In test] attribute value of Someclass.attribute: sentinel.attribute
[In test] attribute value of original: 1
[Outer Test] attribute value of Someclass.attribute: 1
[Outer Test] attribute value of original: 1


위의 예시를 보면 test 범위에서는 Someclass 의 `attribute` 가 `sentinel.attribute` 값으로 변경된 것을 확인할 수 있다. 다만, 다시 밖으로 나와서는 `sentinel.attribute` 가 아닌 기존 값인 `1` 임을 확인할 수 가 있다. 이는 우리가 계속해서 말했던 `patch` 가 지역범위안으로 임시적으로 이뤄지는 것임을 확인할 수 있다.

### sentinel

[**sentinel**](https://docs.python.org/ko/3.13/library/unittest.mock.html#sentinel) 은 무엇일까? sentinel 은 unittest 에서 고유한 attribute 생성시 쉽게 생성하고 이용하게 하기 위한 편의제공을 위해 존재합니다.

예를 들어 우리가 만약 `foo` 라는 attribute 를 만들고 싶을때는 간단하게 아래 처럼 해주면 됩니다.

In [78]:
from unittest.mock import patch, sentinel

sentinel.foo

sentinel.foo

In [79]:
sentinel.bar

sentinel.bar

In [80]:
sentinel.foo == sentinel.bar

False

In [88]:
sentinel.foo == sentinel.foo

True

**그래서 언제 사용하면 좋을까?**

- 특정 객체가 반환 값으로 이용될때 => 고유한 값을 이용하므로 조금 더 트래킹 하기 좋다. (테스트에서 잘 반환되는지를 쉽게 테스트 할수 있음, sentinel 이라는 고유값으로 테스트 하므로)

In [89]:
obj = sentinel.some_object

mock = MagicMock()
mock.method.return_value = obj

mock.method() == obj

True

음, 도대체 어디에 써야하는건지 감이 잘 안올수가 있음. patch 를 mocking 할때 쓸수도 있지만 아래와 같은 상황에도 이용가능함.

In [97]:
class RoachExternalApi:
  api_key = None
  ai_key = 1

  def is_api_key_set(self) -> bool:
    return self.api_key is not None

In [99]:
original_value = RoachExternalApi.api_key
print(f"[OUTER_TEST] original_value: {original_value}")

@patch.object(RoachExternalApi, 'ai_key', None)
def test():
  """
  원래의 값이 None 이라면 예를 들면 위의 상황은 반드시 통과하게 되어 있음/
  왜냐면 우리는 None 만 체크할 수 있기 때문임
  """
  print(f"[IN TEST] original_value: {original_value}")
  assert RoachExternalApi().is_api_key_set() == False
  assert RoachExternalApi.api_key == None

test()
print(f"[OUTER_TEST] original_value: {original_value}")

[OUTER_TEST] original_value: None
[IN TEST] original_value: None
[OUTER_TEST] original_value: None


In [101]:
original_value = RoachExternalApi.api_key
print(f"[OUTER_TEST] original_value: {original_value}")

@patch.object(RoachExternalApi, 'ai_key', sentinel.api_key)
def test():
  """
  하지만 sentinel.api_key 를 이용하면 기존에 의도한 속성으로 제대로 대치되었는지 확인가능함.
  """
  print(f"[IN TEST] original_value: {original_value}")
  assert RoachExternalApi().is_api_key_set() == False
  assert RoachExternalApi.api_key == sentinel.api_key # error 발생

test()
print(f"[OUTER_TEST] original_value: {original_value}")

[OUTER_TEST] original_value: None
[IN TEST] original_value: None


AssertionError: 

조금은 억지스러운 테스트라고 생각할수도 있지만, sentinel.api_key 를 이용하면 기존에 의도한 속성으로 제대로 대치되었는지 확인가능함. 이렇게 속성값이 잘 대치되는지 확인해야 하는 부분에는 `sentinel` 을 이용한 고유값을 이용하는게 좋음.